In [1]:
import cvxpy as cp
import numpy as np

In [ ]:
# Definir variáveis k, f
k, f = 1, 1
# Definir as matrizes A, B1, B2, C e D
A = np.array([[0, 0, 1, 0],
              [0, 0, 0, 1],
              [-k, k, -f, f],
              [k, -k, f, -f]])

B1 = np.array([[0], [0], [0], [1]])

B2 = np.array([[0], [0], [0], [1]])

C = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 0, 0]])

D = np.array([[0], [0], [1]])

n1, n2 = A.shape[0], C.shape[0]

# Definir as variáveis de otimização
W = cp.Variable((n1, n1), symmetric=True)
X = cp.Variable((n2, n2), symmetric=True)
Z = cp.Variable((1, n1))
rho = cp.Variable()

# Blocos para LMI1
M11 = X
M12 = C @ W + D @ Z
M21 = W @ C.T + Z.T @ D.T
M22 = W

# Blocos para LMI2
N11 = A @ W + B2 @ Z + W @ A.T + Z.T @ B2.T
N12 = B1
N21 = B1.T
N22 = -np.eye(B1.shape[1])  # Shape agora é (1, 1)

# Construir as matrizes de blocos LMI
LMI1 = cp.bmat([
    [M11, M12],
    [M21, M22]
])

LMI2 = cp.bmat([
    [N11, N12],
    [N21, N22]
])

# Definir a função objetivo: min (rho)
objective = cp.Minimize(rho)

# Definir as restrições
constraints = [cp.trace(X) <= rho,
               LMI1 >> 0,
               LMI2 << 0]

# Criar e resolver o problema de otimização
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCS) # É uma boa prática especificar o solver

# Verificar o status da solução
if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
    print("Status:", problem.status, "\n")
    print("Valor mínimo de Tr(X):")
    print(problem.value)
    print("\nMatriz W:")
    print(W.value)
    print("\nMatriz X:")
    print(X.value)
    print("\nVetor Z:")
    print(Z.value)
else:
    print("O problema não tem uma solução ótima. Status:", problem.status)


In [7]:
M21

Expression(AFFINE, UNKNOWN, (4, 3))